In [1]:
import numpy as np
import basic
import camb
import os
import cosmology
from matplotlib.pyplot import *
from camb import model, initialpower
from camb.sources import GaussianSourceWindow, SplinedSourceWindow
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

Using CAMB 1.3.2 installed at /global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/camb


In [2]:
cpmodel = 'modelw'
H0    = 70.0
ombh2 = 0.02254
omch2 = 0.11417
As    = 2.41e-09
ns    = 0.97
h0    = H0/100.
Om    = (omch2+ombh2)/h0**2
cps   = {'H0':H0,'Om':Om,'Ov':1-Om,'w0':-1,'wa':0.}

In [3]:
lmin = 1
lmax = 2048
L = np.linspace(0,lmax,lmax+1)

In [4]:
pars = camb.CAMBparams()
pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2)
pars.InitPower.set_params(As=As, ns=ns)
pars.set_for_lmax(lmax, lens_potential_accuracy=5)
pars.NonLinear = model.NonLinear_both
pars.Accuracy.AccuracyBoost = 1
pars.Accuracy.lSampleBoost = 1

In [6]:
# Set matter power spectrum at z: P(k,z)
#pars.set_matter_power(redshifts=[0.],kmax=2e1/h0*1.1,k_per_logint=200)
#results = camb.get_results(pars)
#k, _, pk0 = results.get_linear_matter_power_spectrum(have_power_spectra=True,nonlinear=False)

In [7]:
k, pk0 = np.loadtxt( 'data/'+cpmodel+'/Pk/Pklin.dat', unpack=True )

In [9]:
zcmb = 1088.69

In [10]:
for zn, zsi in [(10,.5),(20,1.),(25,2.),(30,4.),(100,zcmb)]:
    zs = [zsi,zsi,zsi]
    zmin, zmax = 0.0001, min(40.,zsi)
    z, dz = basic.bispec.zpoints(zmin,zmax,zn)
    chi = basic.cosmofuncs.dist_comoving(z,**cps)
    
    # clkk
    clkk = basic.bispec.cl_flat(cpmodel,z,dz,zs[:2],lmax,k,pk0)

    #Lmaxs = np.array([100,500,1000,2000,3000])
    Lmaxs = np.array([100,200,300,400,500])
    snr = np.zeros(len(Lmaxs))

    for i, Lmax in enumerate(Lmaxs):
    
        nlkk = np.zeros(Lmax+1)
        #nlkk[2:] = np.loadtxt(D+'nldd/advact_s6_t1_rlmax4000.dat',unpack=True)[1][:lmax-1]
        nlkk[2:] = np.loadtxt('data/nldd/so/kappa_deproj0_sens2_16000_lT30-3000_lP30-5000.dat',unpack=True)[7][:Lmax-1]
        ckk = clkk[:Lmax+1] + nlkk
        
        snr[i] = basic.bispec.bispeclens_snr(cpmodel,'RT',z,dz,zs,2,Lmax,ckk,k,pk0,btype='kkk')
        print('zs=',zsi,'Lmax=',Lmax,'snr=',np.around(snr[i],decimals=4))
    np.savetxt('data/'+cpmodel+'/snr/snr_kkk_zclean_'+str(zsi)+'_kappa_deproj0_sens2_16000_lT30-3000_lP30-5000.dat',np.array((Lmaxs,snr)).T)

kmax 3.530914825434184
0.5 100 1.597
kmax 7.061829650868368
0.5 200 1.9304
kmax 6.609457250845242
1.0 100 2.431
kmax 13.218914501690485
1.0 200 3.5511
kmax 5.144776102484533
2.0 100 2.4175
kmax 10.289552204969066
2.0 200 4.436
kmax 4.910644380528085
3.0 100 2.1712
kmax 9.82128876105617
3.0 200 4.386
kmax 1.030302654721329
1088.69 100 1.0735
kmax 2.060605309442658
1088.69 200 2.3472
